In [ ]:
import json
import pandas as pd
import numpy as np
import re
import os

In [ ]:
file_dir = "E:\Master_Folder\Movies_ETL"

#f'{file_dir}filename' to open the file

#open and load file from directory
with open(f'{file_dir}\wikipedia-movies.json', mode = 'r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
len (wiki_movies_raw)

In [ ]:
#first 5 records
wiki_movies_raw[:5]

In [ ]:
#last 5 records 
wiki_movies_raw[-5:]

In [ ]:
#checking for the middle of the dataset
wiki_movies_raw[3600:3605]

In [ ]:
#extracting the data into a list
kaggle_metadata = pd.read_csv(f'{file_dir}\movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}\\ratings.csv')

In [ ]:
#instead of using the head() or tail() method... use sample(n= x amount of sample) to get a randomized sample 
kaggle_metadata.sample(n = 5)

In [ ]:
ratings.sample(n=5)

In [ ]:
#transforming wiki_movies series into a dataframe
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

wiki_movies_df.head()

In [ ]:
#listing all of the columns on a list... since .columns would still not display all of the colums due to the high number
wiki_movies_df.columns.tolist()

In [ ]:
#filtering out the movies for ones with 'Director', 'Directed by' or 'imdb_link'
wiki_movies = [movie for movie in wiki_movies_raw 
               if ('Director' in movie or 'Directed by' in movie) 
               and 'imdb_link' in movie
               and 'No. of episodes' not in movie]

#creating a dataframe after the filter
wiki_movies_df = pd.DataFrame(wiki_movies)

wiki_movies_df.head()

In [ ]:
columns_list = wiki_movies_df.columns.tolist()

columns_list

In [ ]:
#A FUNCTION TO CLEAN THE MOVIE DATA
def clean_movie(movie):
    movie = dict(movie) #create a none-destructive copy

    #Look for and add alternate titles into a dictionary
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:

        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key) #remove a key-value pair from a dictionary

    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    #function to change column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)

    #changing the column titles to be consistent throughout
    change_column_name('Directed by', 'Director') 
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    
    return movie

In [ ]:
#making a list of cleaned movies
clean_movies = [clean_movie(movie) for movie in wiki_movies]

#set wiki_movies_df to be the dataframe from the clean_movies, and print out a list of the colums
wiki_movies_df = pd.DataFrame(clean_movies)
clean_col_list = sorted(wiki_movies_df.columns.tolist())

#display columns
clean_col_list

In [ ]:
#removing duplicate rows using regex
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df)) #prints out the original # of rows

#drops any of the duplicate rows
wiki_movies_df.drop_duplicates(subset = 'imdb_id', inplace = True)
print(len(wiki_movies_df)) #prints out the rows w/o duplicates

wiki_movies_df.head()

In [ ]:
#removing mostly null columns

#selecting columns to keep if filled more than 90%
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

#setting the relevant columns to the main dataframe
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [ ]:
#checking for the correct datatypes per column
wiki_movies_df.dtypes

In [ ]:
#drops rows with any missing data values in the 'Box Office' column
box_office = wiki_movies_df['Box office'].dropna()

In [ ]:
#find the values that are not strings in our rows
#only need to be used once so lambda function is created
box_office[box_office.map(lambda x: type(x) != str)]

In [ ]:
#concatenates the lists into one string... there are a few datapoints stored as lists
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
#first form of RegEX to match "$123.4 million/billion." strings
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

#counts the number of matches
box_office.str.contains(form_one, flags = re.IGNORECASE).sum()

In [ ]:
#second form of regex to match "$123,456,789." strings
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

#counts the number of matches
box_office.str.contains(form_two, flags = re.IGNORECASE).sum()

In [ ]:
#finds values that are given as a range
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
#FINDS THE ENTRIES THAT DONT MATCH THE REGEX FROM CREATED

#created two boolean series to select the box office values that dont match either forms
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)

#use pandas elemenet-wise logical operators so no value errors
box_office[~matches_form_one & ~matches_form_two]

In [ ]:
#EXTRACTS AND CONVERTS THE BOX OFFICE VALUES FROM THE FILTERED REGEX FORMS
box_office.str.extract(f'({form_one}|{form_two})')

In [ ]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
#EXTRACT VALUES FROM BOX OFFICES AND APPLY PARSE_DOLLARS FUNCTION TO FIRST COLUMN OF THE DATAFRAME
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [ ]:
#Drop the box office column since no longer needed
wiki_movies_df.drop('Box office', axis =1, inplace= True)

In [ ]:


#creates a 'budget' variable
budget = wiki_movies_df['Budget'].dropna()

#converts any lists to strings
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

#removes any values between a dollar sign and a hyphen(for budgets in given ranges)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex = True)

In [ ]:
#parse the box office data without modifations to the budget data
matches_form_one= budget.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two= budget.str.contains(form_two, flags=re.IGNORECASE)
budget[~matches_form_one & ~matches_form_two]

In [ ]:
#removes the citation references
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]

#30 budget datas remaining... however many of them arent in the right currency so it isnt worth parsing the data for the sake of time

In [ ]:
#parsing the remainder of the budget values
wiki_movies_df['budget'] = budget.str.extract(f'({form_one} | {form_two})', flags= re.IGNORECASE)[0].apply(parse_dollars)

#dropping the original budget column
wiki_movies_df.drop('Budget', axis = 1, inplace = True)

In [ ]:
#PARSE THE RELEASE DATE

#creating a variable that holds non-null values of 'Release Date' in the dataframe
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type == list else x)

In [ ]:
#first form to parse 'month,dd,yyyy'
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'

#second form to parse 'yyyy.mm.dd'
date_form_two = r'\d{4}.[01]\d.[123]\d'

#third form to parse 'month yyyy'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'

#fourth form to parse 'yyyy'
date_form_four = r'\d{4}'

In [ ]:
#extract the dates
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

#parse the filtered dates
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [ ]:
#PARSE RUNNING TIME DATA

#creates a variable to hold non-null values of running time
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x)== list else x)

In [ ]:
#checking for data points with '100 minutes' format
running_time.str.contains(r'^\d*\s*minutes$', flags= re.IGNORECASE).sum()

In [ ]:
#checking for the format of the remainder running time data format
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]

In [ ]:
#checking for other abbreviations of 'minutes' including those that starts with hours
running_time[running_time.str.contains(r'\d*\s*m', flags=re.IGNORECASE) != True]

In [ ]:
#extract digits only
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [ ]:
#converting string values to numerical values
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [ ]:
#convert the hour capture groups and minute capture groups and save the output
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [ ]:
#drop running time fromm the dataset
wiki_movies_df.drop('Running time', axis = 1, inplace= True)

In [ ]:
#checking columns for the correct datatypes
kaggle_metadata.dtypes

In [ ]:
#checking 'adult' columns are either true/false
kaggle_metadata['adult'].value_counts()

In [ ]:
#dont want to include 'adult' videos in a hackathon therefore it will be dropped
#only keep when 'adult' == False
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')

In [ ]:
#checking 'video' columns are either true/false
kaggle_metadata['video'].value_counts()

In [ ]:
#converting the video column into a boolean
kaggle_metadata['video'] == 'True'

#assign the created boolean column above back to 'video' column
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

In [ ]:
#converting 'budget', 'popularity', and 'id' to numeric

#used as.type() method since 'budget' is already in numeric but still needs to be converted to another numeric int
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)

kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors= 'raise') #raise errors to know if there are any datas that cant be converted to numeric


kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors= 'raise') #raise errors to know if there are any datas that cant be converted tp numeric

In [ ]:
#converting 'release date' to datetime
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

In [ ]:
#checking the ratings columns
ratings.info(null_counts= True)

In [ ]:
#converting the 'timestamp' column into unix epoch making the units in seconds
pd.to_datetime(ratings['timestamp'], unit= 's')

In [ ]:
#checkings the 'ratings' column using a histogram to check how dataset is distributed
pd.options.display.float_format = '{:20,.2f}'.format
ratings['rating'].plot(kind='hist')
ratings['rating'].describe()

In [ ]:
movies_df = pd.merge(wiki_movies_df, kaggle_metadata)